In [1]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.stats import norm, sem, t
import matplotlib.mlab as mlab
import re
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

In [4]:
def RunRandomForest(df, question, features, feature_subset):
    
    df['is_train'] = np.random.uniform(0, 1, len(df)) <= .5
    
    for ID in feature_subset.id:
    
        new_header = ID + 'feature'
        df[new_header] = pd.factorize(df[ID])[0]

    # Create two new dataframes, one with the training rows, one with the test rows
    train, test = df[df['is_train']==True], df[df['is_train']==False]

    # train[Question] contains the actual species names. Before we can use it,
    # we need to convert each species name into a digit. So, in this case there
    # are three species, which have been coded as 0, 1, or 2.
    y = pd.factorize(train[question])[0]

    # Create a random forest classifier. By convention, clf means 'classifier'
    clf = RandomForestClassifier(n_jobs=4,n_estimators=128);

    # Train the classifier to take the training features and learn how they relate
    # to the training y (the species)
    clf.fit(train[features], y);

    # Apply the classifier we trained to the test data (which, remember, it has never seen before)
    clf.predict(test[features]);

    # View a list of the features and their importance scores
    return train[features], clf.feature_importances_

def RepeatedRandomForests(df, question, filename, feature_subset, iterations=100):
    
    locs = ((df[question] != 'Valid Skip') | (df[question] != 'NA')) 
    
    # Create a list of the feature column's names
    features = df[locs].columns[189:]

    importance_measures = np.zeros(len(features))

    start = time.clock();

    for i in np.arange(0,iterations):
        
        trained, importance = RunRandomForest(df, question, features, feature_subset);
    
        importance_measures = np.vstack((importance_measures, importance))

    time_elapsed = time.clock() - start;

    print(time_elapsed, 's')

    fd = open(filename,'ab')
    np.savetxt(fd, importance_measures, delimiter=',')
    fd.close()

    return trained, importance_measures, features

def mean_confidence_interval(data, confidence=0.95):
    
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), sem(a)
    h = se * t._ppf((1+confidence)/2., n-1)
    return m, h

def checkFeature(factor):
    
    feature = rel_importance[factor]
    weights = np.ones_like(feature)/float(len(feature))

    fig = plt.figure(num=None, figsize=(4, 3), dpi=120, 
               facecolor='w', edgecolor='k')
    
    ax = fig.add_subplot(111)
    
    n, bins, patches = plt.hist(feature, 20, weights=weights,
                                facecolor='green', alpha=0.75)
    
    plt.title("Distribution for " + factor)
    plt.xlabel("Relative Importance")
    plt.ylabel("Frequency")
    ax.set_xlim((0,0.25))
    
def plotImportance(rel_importance, features, questions, question, iterations=100):
    
    fig = plt.figure(num=None, 
               figsize=(8, 12), 
               dpi=120, 
               facecolor='w', 
               edgecolor='k')
    
    ax = fig.add_subplot(111)

    importances = rel_importance.mean();
    mean, ci = mean_confidence_interval(rel_importance);
    indices = np.argsort(importances);
    
    barlist = plt.barh(range(len(indices)), importances[indices], 
             color='g', align='center',
             xerr = ci[indices])
        
    tags = []
    
    for i in range(len(features)):
        tags.append(questions[questions['feature'] == features[indices][i]]['shortname'].values[0])
    plt.yticks(range(len(indices)), tags)

    #plt.yticks(range(len(indices)), features[indices])
    plt.xlabel('Relative Importance')
    plt.title('Important Features for ' + question)
    
    label = 'Iterations: ' + str(iterations)
    
    bbox_props = dict(boxstyle="round4,pad=0.3", fc="white", lw=1)
    t = ax.text(0.18, 0.12, label, ha="center", va="center", rotation=0, size=12, bbox=bbox_props)
    
def plotCumuluativeImportance(rel_importance, features, questions, iterations=100):
    
    importances = rel_importance.mean()
    mean, ci = mean_confidence_interval(rel_importance)
    indices = np.argsort(importances)[::-1]

    fig = plt.figure(num=None, 
                   figsize=(10, 3), 
                   dpi=120, 
                   facecolor='w', 
                   edgecolor='k')

    ax = fig.add_subplot(111)


    cumulative = np.cumsum(importances[indices])
    
    tags = []
    
    for i in range(len(features)):
        tags.append(questions[questions['feature'] == features[indices][i]]['shortname'].values[0])
    
    plt.errorbar(np.arange(0,len(cumulative)), cumulative, yerr=ci[indices])
    plt.title('Cumulative Importance')
    plt.ylabel('Importance (arb. units)')
    plt.xlabel('Feature')
    plt.xticks(range(len(cumulative)), tags, rotation=90);
    
    label = 'Iterations: ' + str(iterations)
    
    bbox_props = dict(boxstyle="round4,pad=0.3", fc="white", lw=1)
    t = ax.text(2, 0.9, label, ha="center", va="center", rotation=0, size=12, bbox=bbox_props)

In [2]:
folder='../survey_data/';
XLS = pd.ExcelFile(folder + 'LabeledComputationData.xlsx')
XLS2 = pd.ExcelFile(folder + 'CategorizedQuestions.xlsx')
df = XLS.parse('LabeledComputationData')
questions = XLS2.parse('Sheet1')

In [6]:
question = 'Q1';
filename = '../analyzed_data/TeachCompTest100.csv';
iterations = 100;

feature_subset = questions[(questions['scale'] == 'department') & 
          (questions['qtype'] !='open') & 
          (questions['context'] == 'background')];# | (questions['context'] == 'demographics'))];

In [ ]:
trained, feature_importances, features = RepeatedRandomForests(df, question, filename, feature_subset, iterations);
rel_importance = pd.read_csv(filename, names=trained);

In [ ]:
plotImportance(rel_importance, features, questions, question, iterations)

In [ ]:
plotCumuluativeImportance(rel_importance, features, questions, iterations)

In [7]:
sns.heatmap(df[feature_subset].isnull(),yticklabels=False)

ValueError: Must pass DataFrame with boolean values only

In [8]:
print(feature_subset)

       id      feature       scale     context   qtype  \
184  Q185  Q185feature  department  background  binary   
185  Q186  Q186feature  department  background  scaled   
186  Q187  Q187feature  department  background  scaled   

                  shortname     statement  
184             HBCU status      HBCUfact  
185        Institution type  ModClassfact  
186  Highest degree offered   Phy.Degfact  


In [ ]:
locs = ((df[question] != 'Valid Skip') | (df[question] != 'NA')) 

In [ ]:
print(locs)

In [ ]:
df[locs].head()